In [1]:
import pandas as pd
import os

In [2]:
# Voy a integrar el area al maestro de establecimiento, con el objetivo de agregarle mas informacion y poder
# distinguir entre las distintas ares.

In [3]:
# Paths
name_establecimientos_fundadora = '../input_bauti/ultima_campaña/Establecimientos Fundadora.xlsx'
name_maestro_establecimiento = '../Input_fijo/maestro_establecimientos.xlsx'
name_listado_lotes = '../input_bauti/ultima_campaña/Listado de lotes.xlsx'

# Output
path_output = '../input_bauti/maestro_establecimieto/'
name_output = 'maestro_establecimiento_areas.xlsx'

In [4]:
# Load
df_maestro_establecimiento = pd.read_excel(name_maestro_establecimiento)
df_fundadora = pd.read_excel(name_establecimientos_fundadora)
df_lotes =  pd.read_excel(name_listado_lotes)

### Acomodo campos

In [5]:
# Establecimientos fundadora.

# Dropea columnas que posean nullos ('Unnamed')
df_fundadora.dropna(axis=1, how='all', inplace=True)
# Dropeo fillas que contienen algun nulo.
df_fundadora.dropna(axis=0, how='any', inplace=True)
# Dropeo duplicados.
df_fundadora.drop_duplicates(inplace=True)

# Selecciono campos de utilidad.
df_fundadora = df_fundadora[['Zona', 'Field Master Data', 'Good Supplier']]
# Renombro los campos.
df_fundadora.columns = ['zona', 'establecimiento', 'good_supplier']

# Llevo establecimientos a minuscula.
df_fundadora['establecimiento'] = df_fundadora['establecimiento'].str.lower()

# Agrego area.
df_fundadora['area'] = 'fundadora'

print('Shape fundadora - %s Filas / %s Columnas' %(df_fundadora.shape[0], df_fundadora.shape[1]))
df_fundadora.sample(3)

Shape fundadora - 29 Filas / 4 Columnas


,zona,establecimiento,good_supplier,area
13,G,mira sierras 2,932926,fundadora
24,T,la inesita,879005,fundadora
16,H,los varones,932925,fundadora


In [6]:
# Listado de lotes.

# Selecciono campos de utilidad.
df_lotes = df_lotes[['Zona', 'Field Master Data', 'Good Supplier (SAP)', 'Area']]
# Renombro los campos.
df_lotes.columns = ['zona', 'establecimiento', 'good_supplier', 'area']

# Ignoro zonas con nombre "AT".
mask_no_AT = df_lotes['zona']!='AT'
df_lotes = df_lotes.loc[mask_no_AT, :]

# Elimino subzonas que pudieran llegar a aparecer.
df_lotes['zona'] = df_lotes['zona'].str.replace(r'\d*','', regex=True)

# Agrupo por zona, establecimiento y area para obtener registros unicos en base a esos campos.
df_lotes = df_lotes.groupby(['zona', 'establecimiento', 'area']).value_counts().reset_index().drop(columns=0)
df_lotes = df_lotes[['zona', 'establecimiento', 'good_supplier', 'area']]

# Llevo establecimientos y el area a minuscula.
df_lotes['establecimiento'] = df_lotes['establecimiento'].str.lower()
df_lotes['area'] = df_lotes['area'].str.lower()


print('Shape Listado Lotes - %s Filas / %s Columnas' %(df_lotes.shape[0], df_lotes.shape[1]))
df_lotes.sample(3)

Shape Listado Lotes - 165 Filas / 4 Columnas


,zona,establecimiento,good_supplier,area
79,J,san patricio,878956,comercial
46,F,san jorge,922673,comercial
163,Z,quinihua,932454,comercial


### Realizo un merge por si hay establecimientos fundadora que no estan siendo considerados.

In [7]:
df_lotes = df_lotes.merge(df_fundadora, on=['zona', 'establecimiento', 'good_supplier', 'area'], how='left')

print('Shape Listado Lotes - %s Filas / %s Columnas' %(df_lotes.shape[0], df_lotes.shape[1]))
df_lotes.sample(3)

Shape Listado Lotes - 165 Filas / 4 Columnas


,zona,establecimiento,good_supplier,area
134,U,la neja,878930,comercial
71,J,la argentina,922679,comercial
83,L,la gramilla,878763,comercial


### Obtengo un nuevo maestro de establecimientos.

Diferenciado por zona y establecimiento, pero manteniendo las coordenadas del maestro de establecimientos.

In [8]:
print('Shape Maestro Establecimiento - %s Filas / %s Columnas' %(df_maestro_establecimiento.shape[0], df_maestro_establecimiento.shape[1]))
df_maestro_establecimiento.sample(3)

Shape Maestro Establecimiento - 163 Filas / 4 Columnas


,establecimiento,latitud,longitud,Duplicado
121,ojo de agua,-37.977245,-58.012072,0
70,la adelia,-37.675047,-58.593147,0
89,los sauces,-33.900007,-60.708386,0


In [9]:
df_maestro_establecimiento = df_maestro_establecimiento.merge(df_lotes, on='establecimiento', how='inner')

# Ordeno df.
df_maestro_establecimiento = df_maestro_establecimiento[['zona', 'establecimiento', 'good_supplier', 'area', 'latitud', 'longitud', 'Duplicado']]
print('Shape Maestro Establecimiento - %s Filas / %s Columnas' %(df_maestro_establecimiento.shape[0], df_maestro_establecimiento.shape[1]))
df_maestro_establecimiento.sample(3)

Shape Maestro Establecimiento - 165 Filas / 7 Columnas


,zona,establecimiento,good_supplier,area,latitud,longitud,Duplicado
110,U,la américa,878939,comercial,-30.927735,-63.908211,0
86,Y,finca tolloche,878903,comercial,-25.472778,-63.665833,0
56,J,san patricio,878956,comercial,-34.283647,-60.097014,0


In [10]:
# Creo una nueva carpeta para este archivo y lo guardo.
if not os.path.exists(path_output):
    os.makedirs(path_output)

df_maestro_establecimiento.to_excel(path_output+name_output, index=False)